In [ ]:
%pip install python-dotenv langchain langchain-openai

In [2]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser

load_dotenv('../.env')
model = ChatOpenAI(model='gpt-3.5-turbo', temperature=0)

with open('../task/snippets/string-anagram.py') as file:
    code = file.read()
language = 'python'

In [3]:
# Identical prompts to /web/api/src/lib/agent.ts
control_chain = RunnablePassthrough.assign(
    history=lambda inputs: [message for turn in inputs['history'] for message in [('user', turn['input']), ('ai', turn['response'])]]
) | RunnablePassthrough.assign(
    response=ChatPromptTemplate.from_messages([
    ('system', '''You are a helpful assistant. The user is given a code snippet which they have to understand. Note that the user did not write the code or provide the code snippet themselves.
        
Code: """{language}
{code}
"""

Produce an appropriate response to the user's input.'''),
        MessagesPlaceholder('history'),
        ('user', '{input}'),
    ]) | model | StrOutputParser()
)

tom_chain = RunnablePassthrough.assign(
    history=lambda inputs: [message for turn in inputs['history'] for message in [('user', turn['input']), ('ai', turn['response'])]]
) | RunnablePassthrough.assign(
    questions=ChatPromptTemplate.from_messages([
        ('system', '''You are a helpful assistant. The user is given a code snippet which they have to understand. Note that the user did not write the code or provide the code snippet themselves.
            
Code: """{language}
{code}
"""'''),
        MessagesPlaceholder('history'),
        ('user', '{input}'),
        ('user', '''Identify what aspects of the user's mental state are directly relevant to producing a response to the user's input. Phrase these aspects as open-ended questions. The questions should be different enough from each other to each provide valuable insights. Make sure the questions are about the user's mental state and are not leading.''')
    ]) | model | StrOutputParser()
) | RunnablePassthrough.assign(
    mental_state=ChatPromptTemplate.from_messages([
        ('system', '''You are a helpful assistant. The user is given a code snippet which they have to understand. Note that the user did not write the code or provide the code snippet themselves.
            
Code: """{language}
{code}
"""'''),
        MessagesPlaceholder('history'),
        ('user', '{input}'),
        ('user', '''Take the perspective of the user. Answer the following questions about the user's mental state, and explain how this mental state has led to the user's messages. Note that the user did not write the code or provide the code snippet themselves. Therefore, you cannot base your answers on the code snippet or on any code the user copies and pastes, only on the conversation history. Phrase the answers as independent statements, not as responses to the questions.

Questions: """
{questions}
""".''')
    ]) | model | StrOutputParser()
) | RunnablePassthrough.assign(
    response=ChatPromptTemplate.from_messages([
        ('system', '''You are a helpful assistant. The user is given a code snippet which they have to understand. Note that the user did not write the code or provide the code snippet themselves.
            
Code: """{language}
{code}
"""

Produce an appropriate response to the user's input.'''),
        MessagesPlaceholder('history'),
        ('user', '{input}'),
        ('user', '''You have identified the mental state of the user. Use this mental state to produce an appropriate response. Do not mention how the response is based on the user's mental state.
                
Mental state: """
{mental_state}
"""''')
    ]) | model | StrOutputParser()
) | (lambda output: {
    'questions': output['questions'],
    'mental_state': output['mental_state'],
    'response': output['response']
})

# Generate conversations

In [4]:
import json
import os

user_simulation_chain = RunnablePassthrough.assign(
    history=lambda inputs: [message for turn in inputs['history'] for message in [('user', turn['input']), ('ai', turn['response'])]]
) | ChatPromptTemplate.from_messages([
        ('system', '''You are a helpful assistant. The user is given a code snippet which they have to understand. Note that the user did not write the code or provide the code snippet themselves.
            
Code: """{language}
{code}
"""'''),
    MessagesPlaceholder('history'),
    ('user', '''Produce a new question that the user might send to the assistant, by taking their perspective. Your programming experience level is '{experience_level}', but do not explicitly mention this.''')
]) | ChatOpenAI(model='gpt-4', temperature=0.7) | StrOutputParser()

def generate_conversation(filename, experience_level, code, language, n=5):
    if os.path.exists(filename):
        with open(filename) as file:
            history = json.load(file)
    else:
        history = []

    while len(history) < n:
        input = user_simulation_chain.invoke({
            'experience_level': experience_level,
            'code': code,
            'language': language,
            'history': history,
        })
        
        control = control_chain.invoke({
            'code': code,
            'language': language,
            'history': history,
            'input': input
        })

        tom = tom_chain.invoke({
            'code': code,
            'language': language,
            'history': history,
            'input': input
        })

        history.append({'input': input, 'response': control['response'], 'tom': tom})

    with open(filename, 'w') as file:
        json.dump(history, file, indent=4)

In [17]:
generate_conversation('data/simulated-conversation-advanced.json', 'Quite advanced', code, language, 3)

In [6]:
generate_conversation('data/simulated-conversation-novice.json', 'Absolute beginner', code, language, 3)

In [18]:
for user in ['advanced', 'novice']:
    with open(f'data/simulated-conversation-{user}.json') as file:
        data = json.load(file)

    mental_states_latex = ''
    conversation_latex = ''

    for turn in data:
        mental_states_latex += '\\conversationmessageright{' + turn['input'] + '}\n'
        mental_states_latex += '\\conversationmessageleft{\\textbf{Inferred mental state:}\n\n' + turn['tom']['mental_state'] + '}\n'

        conversation_latex += '\\conversationmessageright{' + turn['input'] + '}\n'
        conversation_latex += '\\conversationmessageleft{\\textbf{Response control:}\n\n' + turn['response'].replace('\n', '\n\n') + '}\n'
        conversation_latex += '\\conversationmessageleft{\\textbf{Response \\approach{}:}\n\n' + turn['tom']['response'].replace('\n', '\n\n') + '}\n'

    with open(f'figures/simulated_conversation_mental_states_{user}.tex', 'w') as file:
        file.write(mental_states_latex)
    
    with open(f'figures/simulated_conversation_{user}.tex', 'w') as file:
        file.write(conversation_latex)
